In [2]:
import sys
sys.path.append("../../helper_functions")
import ethereumetl_utils as eetl
import web3py_utils as wpy
import pandas_utils as pu
sys.path.pop()

import pandas as pd
from dotenv import load_dotenv
import requests
import datetime
load_dotenv()
import os

In [3]:
configs = [
        # ['base_goerli',os.environ.get('BASE_GOERLI_NODE')]
        ['op_goerli',os.environ.get('OP_GOERLI_TEAM_NODE')]
]

max_workers = 8
start_timestamp = pu.convert_text_timestamp_to_int('2023-05-19 00:00:00')
end_timestamp = pu.convert_text_timestamp_to_int('2023-05-20 00:00:00')

In [3]:
# Get Transactions
for i in configs:

    folder = i[0]
    uri = i[1]
#     block_arr = wpy.get_blockrange_by_timedelta(i[1],trailing_hours,'hours')
    st_bl = wpy.getBlockByTimestamp(i[1],start_timestamp)# block_arr[0]
    en_bl = wpy.getBlockByTimestamp(i[1],end_timestamp) # block_arr[1]
    print(st_bl)
    print(en_bl)
    eetl.get_ethereum_etl(st_bl,en_bl,folder,uri, max_workers)

9520366
9541803
ethereumetl export_blocks_and_transactions --start-block 9520366 --end-block 9541803 --blocks-output downloads/op_goerli/blocks.csv --transactions-output downloads/op_goerli/transactions.csv --provider-uri https://goerli-replica-0.optimism.io/ --max-workers 8


2023-05-19 11:54:35,223 - ProgressLogger [INFO] - Started work. Items to process: 21438.
2023-05-19 11:54:37,339 - ProgressLogger [INFO] - 2200 items processed. Progress is 10%.
2023-05-19 11:54:38,145 - ProgressLogger [INFO] - 4300 items processed. Progress is 20%.
2023-05-19 11:54:39,269 - ProgressLogger [INFO] - 6500 items processed. Progress is 30%.
2023-05-19 11:54:40,110 - ProgressLogger [INFO] - 8600 items processed. Progress is 40%.
2023-05-19 11:54:41,032 - ProgressLogger [INFO] - 10800 items processed. Progress is 50%.
2023-05-19 11:54:41,809 - ProgressLogger [INFO] - 12900 items processed. Progress is 60%.
2023-05-19 11:54:42,509 - ProgressLogger [INFO] - 15100 items processed. Progress is 70%.
2023-05-19 11:54:43,376 - ProgressLogger [INFO] - 17200 items processed. Progress is 80%.
2023-05-19 11:54:44,095 - ProgressLogger [INFO] - 19300 items processed. Progress is 90%.
2023-05-19 11:54:44,959 - ProgressLogger [INFO] - 21438 items processed. Progress is 100%.
2023-05-19 11:

In [4]:
# Get Receipts
for i in configs:
    folder = i[0]
    uri = i[1]
    eetl.get_eth_etl_receipts(folder, uri)

ethereumetl extract_csv_column --input downloads/op_goerli/transactions.csv --column hash --output downloads/op_goerli/transaction_hashes.txt
ethereumetl export_receipts_and_logs --transaction-hashes downloads/op_goerli/transaction_hashes.txt --provider-uri https://goerli-replica-0.optimism.io/ --receipts-output downloads/op_goerli/receipts.csv


2023-05-19 11:56:46,972 - ProgressLogger [INFO] - Started work.
2023-05-19 11:56:50,515 - ProgressLogger [INFO] - 5000 items processed.
2023-05-19 11:56:53,085 - ProgressLogger [INFO] - 10000 items processed.
2023-05-19 11:56:55,556 - ProgressLogger [INFO] - 15000 items processed.
2023-05-19 11:56:58,109 - ProgressLogger [INFO] - 20000 items processed.
2023-05-19 11:57:00,778 - ProgressLogger [INFO] - 25000 items processed.
2023-05-19 11:57:03,311 - ProgressLogger [INFO] - 30000 items processed.
2023-05-19 11:57:05,862 - ProgressLogger [INFO] - 35000 items processed.
2023-05-19 11:57:06,331 - ProgressLogger [INFO] - Finished work. Total items processed: 35904. Took 0:00:19.358697.
2023-05-19 11:57:06,331 - CompositeItemExporter [INFO] - receipt items exported: 35904
2023-05-19 11:57:06,332 - CompositeItemExporter [INFO] - log items exported: 0


### Process Results

In [5]:
opg_df = pd.read_csv(os.getcwd() + '/downloads/op_goerli/transactions.csv')
rec_df = pd.read_csv(os.getcwd() + '/downloads/op_goerli/receipts.csv')
rec_df = rec_df.rename(columns={'transaction_hash':'hash'})
rec_df = rec_df[['hash','l1_gas_used','l1_fee','l1_gas_price','l1_fee_scalar']]

df = opg_df.merge(rec_df,on='hash',how='left')
#REMOVE DSEPOSITS
df = df[df['transaction_type'] != 126]
df['all_calldata_gas'] = df['l1_gas_used']
df['effective_l1_gas_used'] = df['l1_fee'] / df['l1_gas_price']
display(df)

KeyError: 'l1_fee'